In [1]:
!pip -q install fipy nibabel numpy scipy matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.1/446.1 kB 11.2 MB/s eta 0:00:0000:01


In [2]:
# %% [markdown]
# # In-Silico Brain Drug Delivery (Diffusion + PK/PD)
# - A: 2D (optional 3D) diffusion with decay and heterogeneous D (grey/white proxy), point vs. continuous infusion
# - B: PK/PD with 2–3 compartments and flexible dosing schedules (bolus, intermittent, continuous)
# - Evaluation: heatmaps, time-series, coverage %, overdose %, dose-response trade-offs
# - Optimization: grid search (optional simple BO)
# - Outputs: figures + CSVs saved to /kaggle/working/
# - Related work in notebook footer and LaTeX block (FiPy docs + eBrain/virtual brain sims)
#
# Re-run-safe, pure-NumPy by default; uses FiPy if available and selected.

# %% [markdown]
# ## 1) Imports & Globals

# %%
import os, math, json, time, warnings
from dataclasses import dataclass, asdict
from typing import Tuple, Optional, Dict, List, Callable

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel

# Optional libraries (guarded)
try:
    import nibabel as nib
except Exception:
    nib = None

try:
    import fipy as fp
except Exception:
    fp = None

np.random.seed(42)
OUTDIR = "/kaggle/working"
os.makedirs(OUTDIR, exist_ok=True)

# Matplotlib defaults
plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.dpi"] = 140

# %% [markdown]
# ## 2) Configuration

# %%
@dataclass
class DiffusionConfig:
    use_fipy: bool = False               # Set True to use FiPy if installed (NumPy fallback remains available)
    do_3d: bool = False                  # Optional small 3D run (NumPy only)
    nx: int = 128                        # 2D grid size (x)
    ny: int = 128                        # 2D grid size (y)
    nz: int = 32                         # 3D z for the optional run
    Lx: float = 0.032                    # domain length (m), 3.2 cm
    Ly: float = 0.032                    # domain length (m)
    Lz: float = 0.016                    # thickness for 3D (m)
    D_white: float = 5e-10               # diffusion in white matter (m^2/s) (slower)
    D_grey: float = 1.0e-9               # diffusion in grey matter (m^2/s) (faster)
    k_decay: float = 1.0e-4              # first-order elimination (1/s)
    t_final: float = 3600.0              # total sim time (s) = 1 h
    dt: float = 0.25                     # time step (s) — must satisfy stability for explicit FD
    bctype: str = "neumann"              # "neumann" zero-flux or "dirichlet" zero-value boundaries
    point_injection: bool = True         # True: bolus at t=0; False: continuous infusion
    dose_amount: float = 1.0             # mol/m^3 added as peak at source for point-injection
    infusion_rate: float = 0.001         # mol/(m^3·s) per cell in source region for continuous
    infusion_t_on: float = 0.0           # s
    infusion_t_off: float = 1800.0       # s
    source_radius_frac: float = 0.02     # fraction of domain diag used for source radius
    target_radius_frac: float = 0.10     # fraction of domain diag used for target region
    therapeutic_thresh: float = 0.05     # mol/m^3 threshold for “effective”
    toxicity_thresh: float = 0.5         # mol/m^3 threshold for “overdose”
    use_nifti_mask: bool = False         # if True and nibabel available, load a mask volume
    nifti_path: Optional[str] = None     # path to NIfTI defining tissue (0 white, 1 grey), auto-resampled to grid

@dataclass
class PKPDConfig:
    # Compartments: central/blood (Cb), brain (Cbr), clearance (Cl) optional
    three_compartment: bool = True
    # Rates (1/s)
    k_blood_to_brain: float = 8e-4
    k_brain_to_blood: float = 5e-4
    k_elim_blood: float = 3e-4
    k_deg_brain: float = 2e-4
    k_blood_to_clear: float = 0.0        # if three_compartment True, this is used
    # Volumes (L); we treat concentrations in consistent arbitrary units
    V_blood: float = 5.0
    V_brain: float = 1.2
    V_clear: float = 2.0
    # Dosing (supports bolus list + continuous infusion over windows)
    boluses: List[Tuple[float, float]] = None  # list of (time_s, amount) where amount in “concentration*volume” units
    infusions: List[Tuple[float, float, float]] = None  # list of (t_on, t_off, rate_per_s) in “conc*vol per second”
    t_final: float = 24*3600.0
    # PD
    EC50: float = 0.2
    Emax: float = 1.0
    hill_n: float = 2.0
    therapeutic_thresh: float = 0.1
    toxicity_thresh: float = 1.0

@dataclass
class OptimizeConfig:
    enable_pkpd_grid: bool = True
    enable_diffusion_grid: bool = True
    # PK/PD grid
    bolus_amounts: List[float] = None
    bolus_intervals_h: List[float] = None
    n_bolus: int = 3
    infusion_rates: List[float] = None
    infusion_hours: List[float] = None
    # Diffusion grid (continuous infusion only here)
    dif_infusion_rates: List[float] = None
    dif_infusion_durations: List[float] = None
    # Objective weights
    w_coverage: float = 1.0
    w_overdose: float = 2.0
    w_total_dose: float = 0.1
    try_bo: bool = False   # optional simple BO via sklearn GP

def default_configs():
    diff = DiffusionConfig()
    pkpd = PKPDConfig(
        three_compartment=True,
        boluses=[(0.0, 0.5)],  # 0.5 (conc*vol) at t=0
        infusions=[(2*3600.0, 6*3600.0, 5e-5)]  # from 2h to 6h at small rate
    )
    opt = OptimizeConfig(
        bolus_amounts=[0.25, 0.5, 1.0],
        bolus_intervals_h=[4.0, 6.0, 8.0],
        n_bolus=3,
        infusion_rates=[0.0, 2e-5, 5e-5, 1e-4],
        infusion_hours=[2.0, 4.0, 8.0],
        dif_infusion_rates=[5e-4, 1e-3, 2e-3],
        dif_infusion_durations=[600.0, 1800.0, 3600.0],
    )
    return diff, pkpd, opt

diff_cfg, pkpd_cfg, opt_cfg = default_configs()
print("Configs loaded.")

# %% [markdown]
# ## 3) Utility: Masks & D maps

# %%
def make_2d_masks(nx, ny, Lx, Ly, source_radius_frac, target_radius_frac):
    """
    Create:
      - gm_mask (grey matter) as central oval,
      - wm_mask (white matter) as complement,
      - source mask: small disk near left-center,
      - target mask: disk around center-right (e.g., tumor/target zone).
    """
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y, indexing="ij")

    # Central oval ≈ grey matter
    cx, cy = Lx/2, Ly/2
    a, b = 0.35*Lx, 0.35*Ly
    gm_mask = ((X - cx)**2 / a**2 + (Y - cy)**2 / b**2) <= 1.0
    wm_mask = ~gm_mask

    # Source: small disk near (0.25*Lx, 0.5*Ly)
    sx, sy = 0.25*Lx, 0.5*Ly
    diag = math.hypot(Lx, Ly)
    r_src = source_radius_frac * diag
    source_mask = (X - sx)**2 + (Y - sy)**2 <= r_src**2

    # Target: small disk near (0.75*Lx, 0.5*Ly)
    tx, ty = 0.75*Lx, 0.5*Ly
    r_tgt = target_radius_frac * diag
    target_mask = (X - tx)**2 + (Y - ty)**2 <= r_tgt**2

    return gm_mask, wm_mask, source_mask, target_mask, (X, Y)

def build_D_map(gm_mask, wm_mask, D_grey, D_white):
    D = np.where(gm_mask, D_grey, D_white).astype(np.float64)
    return D

# %% [markdown]
# ## 4) Diffusion (NumPy) solver (heterogeneous D, zero-flux or zero Dirichlet)

# %%
def divergence_of_D_gradC(C, D, dx, dy, bctype="neumann"):
    """
    Computes ∇·(D ∇C) with face-averaged D. Zero-flux or zero-value at boundaries.
    C, D shape: (nx, ny)
    """
    C = C.astype(np.float64)
    D = D.astype(np.float64)
    nx, ny = C.shape

    # Neumann: replicate boundary values into ghost cells
    if bctype.lower() == "neumann":
        C_pad = np.pad(C, ((1,1),(1,1)), mode="edge")
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")
    else:  # Dirichlet zero: pad zeros
        C_pad = np.pad(C, ((1,1),(1,1)), mode="constant", constant_values=0.0)
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")

    # Face-centered D (arithmetic mean)
    Dxp = 0.5*(D_pad[2:,1:-1] + D_pad[1:-1,1:-1])  # i+1/2
    Dxm = 0.5*(D_pad[1:-1,1:-1] + D_pad[0:-2,1:-1])# i-1/2
    Dyp = 0.5*(D_pad[1:-1,2:] + D_pad[1:-1,1:-1])  # j+1/2
    Dym = 0.5*(D_pad[1:-1,1:-1] + D_pad[1:-1,0:-2])# j-1/2

    # Gradients
    dCdx_p = (C_pad[2:,1:-1] - C_pad[1:-1,1:-1]) / dx
    dCdx_m = (C_pad[1:-1,1:-1] - C_pad[0:-2,1:-1]) / dx
    dCdy_p = (C_pad[1:-1,2:] - C_pad[1:-1,1:-1]) / dy
    dCdy_m = (C_pad[1:-1,1:-1] - C_pad[1:-1,0:-2]) / dy

    # Fluxes at faces
    Fx_p = Dxp * dCdx_p
    Fx_m = Dxm * dCdx_m
    Fy_p = Dyp * dCdy_p
    Fy_m = Dym * dCdy_m

    # Divergence
    div = (Fx_p - Fx_m)/dx + (Fy_p - Fy_m)/dy
    return div

def simulate_diffusion_numpy(cfg: DiffusionConfig):
    nx, ny = cfg.nx, cfg.ny
    dx, dy = cfg.Lx/(nx-1), cfg.Ly/(ny-1)

    gm_mask, wm_mask, source_mask, target_mask, (X, Y) = make_2d_masks(
        nx, ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac
    )
    if cfg.use_nifti_mask and cfg.nifti_path and nib is not None:
        try:
            img = nib.load(cfg.nifti_path)
            data = img.get_fdata()
            # Simple resample: choose middle slice, scale to nx, ny
            sl = data.shape[2]//2
            vol = data[:,:,sl]
            from skimage.transform import resize
            vol_res = resize(vol, (nx, ny), order=0, preserve_range=True, anti_aliasing=False)
            gm_mask = (vol_res > 0.5)
            wm_mask = ~gm_mask
        except Exception as e:
            print("NIfTI load failed, using synthetic mask.", e)

    D = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white)

    # Stability check for explicit scheme: dt <= c * min(dx,dy)^2 / (4*max(D))
    dt_stable = 0.2 * min(dx, dy)**2 / (4.0 * max(cfg.D_grey, cfg.D_white) + 1e-30)
    if cfg.dt > dt_stable:
        print(f"[WARN] dt={cfg.dt:.3e} > stability≈{dt_stable:.3e}. Capping dt.")
        dt = dt_stable
    else:
        dt = cfg.dt

    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    C = np.zeros((nx, ny), dtype=np.float64)

    # Apply point bolus at t=0 if configured
    if cfg.point_injection:
        # Normalize so that average in source area equals dose_amount
        nsrc = np.count_nonzero(source_mask)
        if nsrc > 0:
            C[source_mask] += cfg.dose_amount
    # Precompute source mask for infusion
    infusion_active = (not cfg.point_injection)
    if infusion_active:
        print("Using continuous infusion in diffusion.")
    # Timeseries trackers
    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []

    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        # Record snapshots
        if step in snap_indices:
            snap_maps[float(f"{t:.3f}")] = C.copy()

        # Metrics
        tgt_vals = C[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(C >= cfg.toxicity_thresh)))
        total_mass.append(float(C.mean()))  # proportional to total mass (since uniform cell vols)

        if step == nsteps:
            break  # finished

        # Source term for continuous infusion
        S = 0.0
        if infusion_active and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            S = cfg.infusion_rate

        # Update
        div = divergence_of_D_gradC(C, D, dx, dy, cfg.bctype)
        C_new = C + dt*(div - cfg.k_decay*C)
        if S != 0.0:
            C_new[source_mask] += S * dt

        # Enforce non-negative
        C = np.maximum(C_new, 0.0)

    # Save time series CSV
    import pandas as pd
    df = pd.DataFrame({
        "time_s": times,
        "avg_target": [avg_target[0]] + avg_target,  # first element duplicated to align lengths
        "coverage_target": [cov_target[0]] + cov_target,
        "frac_overdose": [frac_overdose[0]] + frac_overdose,
        "total_mass_proxy": [total_mass[0]] + total_mass
    }).iloc[:nsteps+1]
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries.csv"), index=False)

    # Save snapshot heatmaps
    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)")
        plt.ylabel("y (mm)")
        fn = f"diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight")
        plt.close()

    # Save a composite figure
    plt.figure()
    plt.plot(df["time_s"]/60.0, df["avg_target"], label="Avg target conc")
    plt.plot(df["time_s"]/60.0, df["coverage_target"], label="Coverage (target ≥ thresh)")
    plt.plot(df["time_s"]/60.0, df["frac_overdose"], label="Overdose fraction")
    plt.xlabel("Time (min)")
    plt.legend()
    plt.title("Diffusion metrics over time")
    plt.savefig(os.path.join(OUTDIR, "diffusion_metrics_timeseries.png"), bbox_inches="tight")
    plt.close()

    return {
        "times": times,
        "df": df,
        "snapshots": snap_maps,
        "target_mask": target_mask,
        "source_mask": source_mask
    }

# %% [markdown]
# ## 5) Diffusion (FiPy) alternative (optional)

# %%
def simulate_diffusion_fipy(cfg: DiffusionConfig):
    if fp is None:
        raise RuntimeError("FiPy not available.")
    # Grid
    mesh = fp.Grid2D(nx=cfg.nx, ny=cfg.ny, dx=cfg.Lx/(cfg.nx-1), dy=cfg.Ly/(cfg.ny-1))
    C = fp.CellVariable(mesh=mesh, name="C", value=0.0)
    # Masks: build using cell centers
    x = mesh.cellCenters[0].value
    y = mesh.cellCenters[1].value
    X = x.reshape(cfg.nx, cfg.ny, order='F')
    Y = y.reshape(cfg.nx, cfg.ny, order='F')
    gm_mask, wm_mask, source_mask, target_mask, _ = make_2d_masks(
        cfg.nx, cfg.ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac
    )
    D_map = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white).astype(np.float64).ravel(order='F')
    D = fp.CellVariable(mesh=mesh, value=D_map)

    if cfg.point_injection:
        C.setValue(cfg.dose_amount, where=source_mask.ravel(order='F'))

    eq = fp.TransientTerm() == fp.DiffusionTerm(coeff=D) - cfg.k_decay * C
    dt = cfg.dt
    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    import pandas as pd
    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []
    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        if step in snap_indices:
            Cmap = C.value.reshape(cfg.nx, cfg.ny, order='F').copy()
            snap_maps[float(f"{t:.3f}")] = Cmap

        arr = C.value.reshape(cfg.nx, cfg.ny, order='F')
        tgt_vals = arr[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(arr >= cfg.toxicity_thresh)))
        total_mass.append(float(arr.mean()))

        if step == nsteps:
            break

        # infusion
        if (not cfg.point_injection) and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            cur = C.value.copy()
            cur[source_mask.ravel(order='F')] += cfg.infusion_rate * dt
            C.setValue(cur)

        eq.solve(var=C, dt=dt)

    df = pd.DataFrame({
        "time_s": times,
        "avg_target": [avg_target[0]] + avg_target,
        "coverage_target": [cov_target[0]] + cov_target,
        "frac_overdose": [frac_overdose[0]] + frac_overdose,
        "total_mass_proxy": [total_mass[0]] + total_mass
    }).iloc[:nsteps+1]
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries_fipy.csv"), index=False)

    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"[FiPy] Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"fipy_diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    return {
        "times": times,
        "df": df,
        "snapshots": snap_maps,
        "target_mask": target_mask,
        "source_mask": source_mask
    }

# %% [markdown]
# ## 6) Optional: tiny 3D diffusion (NumPy, constant D)

# %%
def simulate_diffusion_3d_numpy(cfg: DiffusionConfig):
    nx, ny, nz = cfg.nx//2, cfg.ny//2, cfg.nz
    Lx, Ly, Lz = cfg.Lx, cfg.Ly, cfg.Lz
    dx, dy, dz = Lx/(nx-1), Ly/(ny-1), Lz/(nz-1)
    C = np.zeros((nx, ny, nz), dtype=np.float64)
    D = cfg.D_grey  # uniform for simplicity
    dt = min(cfg.dt, 0.1 * min(dx,dy,dz)**2 / (6*D + 1e-30))
    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    # source: central small sphere
    x = np.linspace(0, Lx, nx); y = np.linspace(0, Ly, ny); z = np.linspace(0, Lz, nz)
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")
    cx, cy, cz = 0.25*Lx, 0.5*Ly, 0.5*Lz
    r = cfg.source_radius_frac * math.sqrt(Lx**2 + Ly**2 + Lz**2)
    src = (X-cx)**2 + (Y-cy)**2 + (Z-cz)**2 <= r**2
    if cfg.point_injection:
        C[src] += cfg.dose_amount

    def laplacian(C):
        Cp = np.pad(C, ((1,1),(1,1),(1,1)), mode="edge")  # Neumann
        d2x = (Cp[2:,1:-1,1:-1] - 2*Cp[1:-1,1:-1,1:-1] + Cp[:-2,1:-1,1:-1])/dx**2
        d2y = (Cp[1:-1,2:,1:-1] - 2*Cp[1:-1,1:-1,1:-1] + Cp[1:-1,:-2,1:-1])/dy**2
        d2z = (Cp[1:-1,1:-1,2:] - 2*Cp[1:-1,1:-1,1:-1] + Cp[1:-1,1:-1,:-2])/dz**2
        return d2x + d2y + d2z

    snapshots = {}
    snap_times = [0.0, cfg.t_final*0.5, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])

    for step, t in enumerate(times):
        if step in snap_indices:
            snapshots[float(f"{t:.3f}")] = C.copy()

        if step == nsteps:
            break

        L = laplacian(C)
        C_new = C + dt*(D*L - cfg.k_decay*C)
        if (not cfg.point_injection) and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            C_new[src] += cfg.infusion_rate * dt
        C = np.maximum(C_new, 0.0)

    # Save mid-slice figures
    for tkey, vol in snapshots.items():
        midz = vol[:,:,nz//2]
        plt.figure()
        plt.imshow(midz.T, origin="lower", extent=[0, Lx*1e3, 0, Ly*1e3], aspect="equal")
        plt.colorbar(label="C")
        plt.title(f"3D mid-slice at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"diffusion3d_slice_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    return {"times": times, "snapshots": snapshots}

# %% [markdown]
# ## 7) PK/PD ODEs + PD linkage

# %%
def pkpd_rhs(t, y, p: PKPDConfig, dose_fn: Callable[[float], float]):
    # y = [Cb, Cbr, Cl] or [Cb, Cbr]
    Cb, Cbr = y[0], y[1]
    input_rate = dose_fn(t)  # in conc*vol per second
    dCb = - (p.k_blood_to_brain + p.k_elim_blood + (p.k_blood_to_clear if p.three_compartment else 0.0)) * Cb \
          + p.k_brain_to_blood * Cbr \
          + input_rate / max(p.V_blood, 1e-12)
    dCbr = p.k_blood_to_brain * Cb - (p.k_brain_to_blood + p.k_deg_brain) * Cbr
    if p.three_compartment:
        Cl = y[2]
        dCl = p.k_blood_to_clear * Cb
        return [dCb, dCbr, dCl]
    else:
        return [dCb, dCbr]

def make_dose_fn(boluses: List[Tuple[float,float]], infusions: List[Tuple[float,float,float]]):
    # bolus: (t, amount); infusion: (t_on, t_off, rate)
    boluses = boluses or []
    infusions = infusions or []
    def dose(t: float) -> float:
        rt = 0.0
        # instantaneous boluses -> approximate as sharp square over 1 second
        for tb, amt in boluses:
            if tb <= t < tb + 1.0:
                rt += amt  # amount per second over 1s
        for t_on, t_off, r in infusions:
            if t_on <= t <= t_off:
                rt += r
        return rt
    return dose

def simulate_pkpd(cfg: PKPDConfig):
    y0 = [0.0, 0.0] if not cfg.three_compartment else [0.0, 0.0, 0.0]
    t_eval = np.linspace(0, cfg.t_final, 1200)
    dose_fn = make_dose_fn(cfg.boluses, cfg.infusions)
    rhs = lambda t, y: pkpd_rhs(t, y, cfg, dose_fn)
    sol = solve_ivp(rhs, [0, cfg.t_final], y0, t_eval=t_eval, method="RK45", rtol=1e-6, atol=1e-9)

    if not sol.success:
        print("[WARN] PK/PD solver returned unsuccessful status.")

    Cb = sol.y[0]
    Cbr = sol.y[1]
    Cl = sol.y[2] if cfg.three_compartment else None

    # PD: Hill
    E = cfg.Emax * (Cbr**cfg.hill_n) / (cfg.EC50**cfg.hill_n + Cbr**cfg.hill_n)

    # KPIs
    dt = np.diff(sol.t, prepend=sol.t[0])
    time_above_ther = float(np.sum((Cbr >= cfg.therapeutic_thresh) * dt))
    time_over_toxic = float(np.sum((Cbr >= cfg.toxicity_thresh) * dt))
    auE = float(np.trapz(E, sol.t))
    total_dose = 0.0
    if cfg.boluses:
        total_dose += sum([amt for _, amt in cfg.boluses])
    if cfg.infusions:
        total_dose += sum([(t_off - t_on) * r for (t_on, t_off, r) in cfg.infusions])

    # Save CSV + plot
    import pandas as pd
    df = pd.DataFrame({
        "time_s": sol.t,
        "Cb": Cb,
        "Cbrain": Cbr,
        "Effect": E
    })
    df.to_csv(os.path.join(OUTDIR, "pkpd_timeseries.csv"), index=False)

    plt.figure()
    plt.plot(sol.t/3600.0, Cb, label="Blood")
    plt.plot(sol.t/3600.0, Cbr, label="Brain")
    plt.axhline(cfg.therapeutic_thresh, ls="--", lw=0.7, label="Therapeutic thresh")
    plt.axhline(cfg.toxicity_thresh, ls="--", lw=0.7, label="Toxicity thresh")
    plt.xlabel("Time (h)"); plt.ylabel("Concentration (arb.)")
    plt.title("PK across compartments")
    plt.legend()
    plt.savefig(os.path.join(OUTDIR, "pkpd_compartments.png"), bbox_inches="tight")
    plt.close()

    plt.figure()
    plt.plot(sol.t/3600.0, E)
    plt.xlabel("Time (h)"); plt.ylabel("Effect (PD, Hill)")
    plt.title("PD effect")
    plt.savefig(os.path.join(OUTDIR, "pkpd_effect.png"), bbox_inches="tight")
    plt.close()

    return {
        "t": sol.t, "Cb": Cb, "Cbrain": Cbr, "Cl": Cl, "Effect": E,
        "time_above_ther": time_above_ther,
        "time_over_toxic": time_over_toxic,
        "auE": auE,
        "total_dose": total_dose
    }

# %% [markdown]
# ## 8) Optimization sketches (grid search + optional BO)

# %%
def score_objective(coverage_mean, overdose_mean, total_dose, w_cov=1.0, w_od=2.0, w_dose=0.1):
    # Larger is better; penalize overdose, dose
    return w_cov*coverage_mean - w_od*overdose_mean - w_dose*total_dose

def grid_search_pkpd(base_cfg: PKPDConfig, opt: OptimizeConfig):
    records = []
    for amt in (opt.bolus_amounts or [0.5]):
        for interval_h in (opt.bolus_intervals_h or [6.0]):
            boluses = [(i*interval_h*3600.0, amt) for i in range(opt.n_bolus)]
            for inf_rate in (opt.infusion_rates or [0.0]):
                for inf_h in (opt.infusion_hours or [0.0]):
                    test = PKPDConfig(**asdict(base_cfg))
                    test.boluses = boluses
                    if inf_h > 0 and inf_rate > 0:
                        test.infusions = [(2*3600.0, (2+inf_h)*3600.0, inf_rate)]
                    else:
                        test.infusions = []
                    res = simulate_pkpd(test)
                    # Approximate coverage/overdose via PK brain conc vs thresholds
                    coverage_mean = float(np.mean(res["Cbrain"] >= test.therapeutic_thresh))
                    overdose_mean = float(np.mean(res["Cbrain"] >= test.toxicity_thresh))
                    score = score_objective(coverage_mean, overdose_mean, res["total_dose"],
                                            opt.w_coverage, opt.w_overdose, opt.w_total_dose)
                    records.append({
                        "bolus_amt": amt,
                        "bolus_interval_h": interval_h,
                        "n_bolus": opt.n_bolus,
                        "inf_rate": inf_rate,
                        "inf_hours": inf_h,
                        "coverage_mean": coverage_mean,
                        "overdose_mean": overdose_mean,
                        "total_dose": res["total_dose"],
                        "score": score
                    })
    import pandas as pd
    df = pd.DataFrame.from_records(records)
    df.sort_values("score", ascending=False, inplace=True)
    df.to_csv(os.path.join(OUTDIR, "pkpd_grid_search_results.csv"), index=False)

    # Frontier plot
    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (mean time fraction ≥ therapeutic)")
    plt.ylabel("Overdose (mean time fraction ≥ toxicity)")
    plt.title("PK/PD trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "pkpd_tradeoff_grid.png"), bbox_inches="tight")
    plt.close()
    return df

def grid_search_diffusion(base_cfg: DiffusionConfig, opt: OptimizeConfig):
    records = []
    for rate in (opt.dif_infusion_rates or [base_cfg.infusion_rate]):
        for dur in (opt.dif_infusion_durations or [base_cfg.infusion_t_off - base_cfg.infusion_t_on]):
            test = DiffusionConfig(**asdict(base_cfg))
            test.point_injection = False
            test.infusion_rate = rate
            test.infusion_t_on = 0.0
            test.infusion_t_off = dur
            out = simulate_diffusion_numpy(test)
            df = out["df"]
            coverage_mean = float(df["coverage_target"].mean())
            overdose_mean = float(df["frac_overdose"].mean())
            total_dose = rate * dur * float(np.count_nonzero(out["source_mask"]))  # rough proxy
            score = score_objective(coverage_mean, overdose_mean, total_dose,
                                    opt.w_coverage, opt.w_overdose, opt.w_total_dose)
            records.append({
                "inf_rate": rate,
                "inf_dur_s": dur,
                "coverage_mean": coverage_mean,
                "overdose_mean": overdose_mean,
                "total_dose_proxy": total_dose,
                "score": score
            })
    import pandas as pd
    df = pd.DataFrame.from_records(records)
    df.sort_values("score", ascending=False, inplace=True)
    df.to_csv(os.path.join(OUTDIR, "diffusion_grid_search_results.csv"), index=False)

    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (spatial fraction ≥ therapeutic)")
    plt.ylabel("Overdose (spatial fraction ≥ toxicity)")
    plt.title("Diffusion trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "diffusion_tradeoff_grid.png"), bbox_inches="tight")
    plt.close()
    return df

def bo_demo_pkpd(base_cfg: PKPDConfig, opt: OptimizeConfig, n_init=6, n_iter=12):
    # Simple BO over [bolus_amt, inf_rate] with fixed interval/length
    space_bounds = np.array([[0.1, 1.5], [0.0, 2e-4]])  # amt, inf_rate
    X = []
    y = []
    kernel = C(1.0, (1e-2, 1e3)) * Matern(nu=1.5) + WhiteKernel(1e-5)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

    def eval_point(amt, inf_rate):
        test = PKPDConfig(**asdict(base_cfg))
        test.boluses = [(0.0, amt), (6*3600.0, amt), (12*3600.0, amt)]
        test.infusions = [(2*3600.0, 6*3600.0, inf_rate)]
        res = simulate_pkpd(test)
        cov = float(np.mean(res["Cbrain"] >= test.therapeutic_thresh))
        od = float(np.mean(res["Cbrain"] >= test.toxicity_thresh))
        score = score_objective(cov, od, res["total_dose"], opt.w_coverage, opt.w_overdose, opt.w_total_dose)
        return score

    # Latin-like init
    for i in range(n_init):
        r = np.random.rand(2)
        x = space_bounds[:,0] + r*(space_bounds[:,1]-space_bounds[:,0])
        X.append(x); y.append(eval_point(x[0], x[1]))

    X = np.array(X); y = np.array(y)
    for it in range(n_iter):
        gp.fit(X, y)
        # EI acquisition on random candidates
        cand = space_bounds[:,0] + np.random.rand(200,2)*(space_bounds[:,1]-space_bounds[:,0])
        mu, sigma = gp.predict(cand, return_std=True)
        best = y.max()
        from scipy.stats import norm
        with np.errstate(divide='ignore'):
            Z = (mu - best) / (sigma + 1e-9)
        ei = (mu - best) * norm.cdf(Z) + sigma * norm.pdf(Z)
        x_new = cand[np.argmax(ei)]
        y_new = eval_point(x_new[0], x_new[1])
        X = np.vstack([X, x_new]); y = np.append(y, y_new)

    import pandas as pd
    df = pd.DataFrame({"bolus_amt": X[:,0], "inf_rate": X[:,1], "score": y})
    df.sort_values("score", ascending=False, inplace=True)
    df.to_csv(os.path.join(OUTDIR, "pkpd_bo_results.csv"), index=False)
    return df

# %% [markdown]
# ## 9) Run: Diffusion, PK/PD, Optimization

# %%
print("Running diffusion (NumPy)...")
dif_out = simulate_diffusion_numpy(diff_cfg)

if diff_cfg.use_fipy and fp is not None:
    print("Running diffusion (FiPy)...")
    _ = simulate_diffusion_fipy(diff_cfg)

if diff_cfg.do_3d:
    print("Running tiny 3D diffusion...")
    _ = simulate_diffusion_3d_numpy(diff_cfg)

print("Running PK/PD...")
pk_out = simulate_pkpd(pkpd_cfg)

print("Running grid searches...")
pk_grid = grid_search_pkpd(pkpd_cfg, opt_cfg) if opt_cfg.enable_pkpd_grid else None
dif_grid = grid_search_diffusion(diff_cfg, opt_cfg) if opt_cfg.enable_diffusion_grid else None

if opt_cfg.try_bo:
    print("Running simple BO demo for PK/PD...")
    _ = bo_demo_pkpd(pkpd_cfg, opt_cfg)

# %% [markdown]
# ## 10) Notebook footer: outputs list & related work

# %%
print("Saved files in:", OUTDIR)
print(sorted(os.listdir(OUTDIR)))

print("\nRelated work:")
print(" - FiPy: A Finite Volume PDE Solver in Python. Docs: fipy.readthedocs.io")
print(" - eBrain / virtual-brain-style in-silico platforms for neuro drug delivery & transport simulations.")

Configs loaded.
Running diffusion (NumPy)...


ValueError: All arrays must be of the same length

In [3]:
# %% [markdown]
# # In-Silico Brain Drug Delivery (Diffusion + PK/PD)
# - A: 2D (optional 3D) diffusion with decay and heterogeneous D (grey/white proxy), point vs. continuous infusion
# - B: PK/PD with 2–3 compartments and flexible dosing schedules (bolus, intermittent, continuous)
# - Evaluation: heatmaps, time-series, coverage %, overdose %, dose-response trade-offs
# - Optimization: grid search (optional simple BO)
# - Outputs: figures + CSVs saved to /kaggle/working/
# - Related work in notebook footer and LaTeX block (FiPy docs + eBrain/virtual brain sims)
#
# Re-run-safe, pure-NumPy by default; uses FiPy if available and selected.
# %% [markdown]
# ## 1) Imports & Globals

# %%
import os, math, json, time, warnings
from dataclasses import dataclass, asdict
from typing import Tuple, Optional, Dict, List, Callable

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Optional libs
try:
    import pandas as pd
except Exception:
    pd = None

try:
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel
except Exception:
    GaussianProcessRegressor = None
    Matern = C = WhiteKernel = None

try:
    import nibabel as nib
except Exception:
    nib = None

try:
    import fipy as fp
except Exception:
    fp = None

np.random.seed(42)
OUTDIR = "/kaggle/working"
os.makedirs(OUTDIR, exist_ok=True)

# Matplotlib defaults
plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.dpi"] = 140

# %% [markdown]
# ## 2) Configuration

# %%
@dataclass
class DiffusionConfig:
    use_fipy: bool = False               # Set True to use FiPy if installed (NumPy fallback remains available)
    do_3d: bool = False                  # Optional small 3D run (NumPy only)
    nx: int = 128                        # 2D grid size (x)
    ny: int = 128                        # 2D grid size (y)
    nz: int = 32                         # 3D z for the optional run
    Lx: float = 0.032                    # domain length (m), 3.2 cm
    Ly: float = 0.032                    # domain length (m)
    Lz: float = 0.016                    # thickness for 3D (m)
    D_white: float = 5e-10               # diffusion in white matter (m^2/s) (slower)
    D_grey: float = 1.0e-9               # diffusion in grey matter (m^2/s) (faster)
    k_decay: float = 1.0e-4              # first-order elimination (1/s)
    t_final: float = 3600.0              # total sim time (s) = 1 h
    dt: float = 0.25                     # time step (s) — must satisfy stability for explicit FD
    bctype: str = "neumann"              # "neumann" zero-flux or "dirichlet" zero-value boundaries
    point_injection: bool = True         # True: bolus at t=0; False: continuous infusion
    dose_amount: float = 1.0             # mol/m^3 added at source for point injection
    infusion_rate: float = 0.001         # mol/(m^3·s) per cell in source region for continuous
    infusion_t_on: float = 0.0           # s
    infusion_t_off: float = 1800.0       # s
    source_radius_frac: float = 0.02     # fraction of domain diag for source radius
    target_radius_frac: float = 0.10     # fraction of domain diag for target region
    therapeutic_thresh: float = 0.05     # mol/m^3 threshold for “effective”
    toxicity_thresh: float = 0.5         # mol/m^3 threshold for “overdose”
    use_nifti_mask: bool = False         # if True and nibabel available, load a mask volume
    nifti_path: Optional[str] = None     # path to NIfTI defining tissue (0 white, 1 grey), auto-resampled to grid

@dataclass
class PKPDConfig:
    three_compartment: bool = True
    # Rates (1/s)
    k_blood_to_brain: float = 8e-4
    k_brain_to_blood: float = 5e-4
    k_elim_blood: float = 3e-4
    k_deg_brain: float = 2e-4
    k_blood_to_clear: float = 0.0
    # Volumes (arbitrary units, consistent)
    V_blood: float = 5.0
    V_brain: float = 1.2
    V_clear: float = 2.0
    # Dosing (boluses + continuous infusions)
    boluses: List[Tuple[float, float]] = None
    infusions: List[Tuple[float, float, float]] = None
    t_final: float = 24*3600.0
    # PD
    EC50: float = 0.2
    Emax: float = 1.0
    hill_n: float = 2.0
    therapeutic_thresh: float = 0.1
    toxicity_thresh: float = 1.0

@dataclass
class OptimizeConfig:
    enable_pkpd_grid: bool = True
    enable_diffusion_grid: bool = True
    # PK/PD grid
    bolus_amounts: List[float] = None
    bolus_intervals_h: List[float] = None
    n_bolus: int = 3
    infusion_rates: List[float] = None
    infusion_hours: List[float] = None
    # Diffusion grid (continuous infusion only)
    dif_infusion_rates: List[float] = None
    dif_infusion_durations: List[float] = None
    # Objective weights
    w_coverage: float = 1.0
    w_overdose: float = 2.0
    w_total_dose: float = 0.1
    try_bo: bool = False   # optional simple BO via sklearn GP

def default_configs():
    diff = DiffusionConfig()
    pkpd = PKPDConfig(
        three_compartment=True,
        boluses=[(0.0, 0.5)],
        infusions=[(2*3600.0, 6*3600.0, 5e-5)]
    )
    opt = OptimizeConfig(
        bolus_amounts=[0.25, 0.5, 1.0],
        bolus_intervals_h=[4.0, 6.0, 8.0],
        n_bolus=3,
        infusion_rates=[0.0, 2e-5, 5e-5, 1e-4],
        infusion_hours=[2.0, 4.0, 8.0],
        dif_infusion_rates=[5e-4, 1e-3, 2e-3],
        dif_infusion_durations=[600.0, 1800.0, 3600.0],
    )
    return diff, pkpd, opt

diff_cfg, pkpd_cfg, opt_cfg = default_configs()
print("Configs loaded.")

# %% [markdown]
# ## 3) Utility: Masks & D maps

# %%
def make_2d_masks(nx, ny, Lx, Ly, source_radius_frac, target_radius_frac):
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y, indexing="ij")

    cx, cy = Lx/2, Ly/2
    a, b = 0.35*Lx, 0.35*Ly
    gm_mask = ((X - cx)**2 / a**2 + (Y - cy)**2 / b**2) <= 1.0
    wm_mask = ~gm_mask

    sx, sy = 0.25*Lx, 0.5*Ly
    diag = math.hypot(Lx, Ly)
    r_src = source_radius_frac * diag
    source_mask = (X - sx)**2 + (Y - sy)**2 <= r_src**2

    tx, ty = 0.75*Lx, 0.5*Ly
    r_tgt = target_radius_frac * diag
    target_mask = (X - tx)**2 + (Y - ty)**2 <= r_tgt**2

    return gm_mask, wm_mask, source_mask, target_mask, (X, Y)

def build_D_map(gm_mask, wm_mask, D_grey, D_white):
    return np.where(gm_mask, D_grey, D_white).astype(np.float64)

# %% [markdown]
# ## 4) Diffusion (NumPy) solver (heterogeneous D, zero-flux or zero Dirichlet)

# %%
def divergence_of_D_gradC(C, D, dx, dy, bctype="neumann"):
    C = C.astype(np.float64)
    D = D.astype(np.float64)

    if bctype.lower() == "neumann":
        C_pad = np.pad(C, ((1,1),(1,1)), mode="edge")
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")
    else:  # Dirichlet zero
        C_pad = np.pad(C, ((1,1),(1,1)), mode="constant", constant_values=0.0)
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")

    Dxp = 0.5*(D_pad[2:,1:-1] + D_pad[1:-1,1:-1])
    Dxm = 0.5*(D_pad[1:-1,1:-1] + D_pad[0:-2,1:-1])
    Dyp = 0.5*(D_pad[1:-1,2:] + D_pad[1:-1,1:-1])
    Dym = 0.5*(D_pad[1:-1,1:-1] + D_pad[1:-1,0:-2])

    dCdx_p = (C_pad[2:,1:-1] - C_pad[1:-1,1:-1])
    dCdx_m = (C_pad[1:-1,1:-1] - C_pad[0:-2,1:-1])
    dCdy_p = (C_pad[1:-1,2:] - C_pad[1:-1,1:-1])
    dCdy_m = (C_pad[1:-1,1:-1] - C_pad[1:-1,0:-2])

    Fx_p = Dxp * dCdx_p
    Fx_m = Dxm * dCdx_m
    Fy_p = Dyp * dCdy_p
    Fy_m = Dym * dCdy_m

    return (Fx_p - Fx_m)/(dx**2) + (Fy_p - Fy_m)/(dy**2)

def simulate_diffusion_numpy(cfg: DiffusionConfig):
    nx, ny = cfg.nx, cfg.ny
    dx, dy = cfg.Lx/(nx-1), cfg.Ly/(ny-1)

    gm_mask, wm_mask, source_mask, target_mask, (X, Y) = make_2d_masks(
        nx, ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac
    )
    if cfg.use_nifti_mask and cfg.nifti_path and nib is not None:
        try:
            img = nib.load(cfg.nifti_path)
            data = img.get_fdata()
            sl = data.shape[2]//2
            vol = data[:,:,sl]
            # Lazy nearest-neighbor resize to avoid extra deps
            xi = (np.linspace(0, vol.shape[0]-1, nx)).astype(int)
            yi = (np.linspace(0, vol.shape[1]-1, ny)).astype(int)
            vol_res = vol[np.ix_(xi, yi)]
            gm_mask = (vol_res > 0.5)
            wm_mask = ~gm_mask
        except Exception as e:
            print("NIfTI load/resample failed, using synthetic mask.", e)

    D = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white)

    dt_stable = 0.2 * min(dx, dy)**2 / (4.0 * max(cfg.D_grey, cfg.D_white) + 1e-30)
    dt = min(cfg.dt, dt_stable)
    if cfg.dt > dt_stable:
        print(f"[WARN] dt={cfg.dt:.3e} > stability≈{dt_stable:.3e}. Using dt={dt:.3e}")

    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    C = np.zeros((nx, ny), dtype=np.float64)
    if cfg.point_injection:
        if np.count_nonzero(source_mask) > 0:
            C[source_mask] += cfg.dose_amount

    infusion_active = (not cfg.point_injection)

    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []
    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        if step in snap_indices:
            snap_maps[float(f"{t:.3f}")] = C.copy()

        tgt_vals = C[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(C >= cfg.toxicity_thresh)))
        total_mass.append(float(C.mean()))

        if step == nsteps:
            break

        S = 0.0
        if infusion_active and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            S = cfg.infusion_rate

        div = divergence_of_D_gradC(C, D, dx, dy, cfg.bctype)
        C_new = C + dt*(div - cfg.k_decay*C)
        if S != 0.0:
            C_new[source_mask] += S * dt

        C = np.maximum(C_new, 0.0)

    # ---- FIXED: arrays are all the same length as `times`
    if pd is None:
        raise RuntimeError("pandas unavailable")
    df = pd.DataFrame({
        "time_s": times,
        "avg_target": avg_target,
        "coverage_target": cov_target,
        "frac_overdose": frac_overdose,
        "total_mass_proxy": total_mass
    })
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries.csv"), index=False)

    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    plt.figure()
    plt.plot(df["time_s"]/60.0, df["avg_target"], label="Avg target conc")
    plt.plot(df["time_s"]/60.0, df["coverage_target"], label="Coverage (target ≥ thresh)")
    plt.plot(df["time_s"]/60.0, df["frac_overdose"], label="Overdose fraction")
    plt.xlabel("Time (min)"); plt.legend(); plt.title("Diffusion metrics over time")
    plt.savefig(os.path.join(OUTDIR, "diffusion_metrics_timeseries.png"), bbox_inches="tight"); plt.close()

    return {
        "times": times,
        "df": df,
        "snapshots": snap_maps,
        "target_mask": target_mask,
        "source_mask": source_mask
    }

# %% [markdown]
# ## 5) Diffusion (FiPy) alternative (optional)

# %%
def simulate_diffusion_fipy(cfg: DiffusionConfig):
    if fp is None:
        raise RuntimeError("FiPy not available.")
    mesh = fp.Grid2D(nx=cfg.nx, ny=cfg.ny, dx=cfg.Lx/(cfg.nx-1), dy=cfg.Ly/(cfg.ny-1))
    Cvar = fp.CellVariable(mesh=mesh, name="C", value=0.0)

    x = mesh.cellCenters[0].value
    y = mesh.cellCenters[1].value
    X = x.reshape(cfg.nx, cfg.ny, order='F')
    Y = y.reshape(cfg.nx, cfg.ny, order='F')

    gm_mask, wm_mask, source_mask, target_mask, _ = make_2d_masks(
        cfg.nx, cfg.ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac
    )
    D_map = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white).astype(np.float64).ravel(order='F')
    D = fp.CellVariable(mesh=mesh, value=D_map)

    if cfg.point_injection:
        Ctmp = Cvar.value.copy()
        Ctmp[source_mask.ravel(order='F')] += cfg.dose_amount
        Cvar.setValue(Ctmp)

    eq = fp.TransientTerm() == fp.DiffusionTerm(coeff=D) - cfg.k_decay * Cvar
    dt = cfg.dt
    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []
    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        if step in snap_indices:
            Cmap = Cvar.value.reshape(cfg.nx, cfg.ny, order='F').copy()
            snap_maps[float(f"{t:.3f}")] = Cmap

        arr = Cvar.value.reshape(cfg.nx, cfg.ny, order='F')
        tgt_vals = arr[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(arr >= cfg.toxicity_thresh)))
        total_mass.append(float(arr.mean()))

        if step == nsteps:
            break

        if (not cfg.point_injection) and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            cur = Cvar.value.copy()
            cur[source_mask.ravel(order='F')] += cfg.infusion_rate * dt
            Cvar.setValue(cur)

        eq.solve(var=Cvar, dt=dt)

    if pd is None:
        raise RuntimeError("pandas unavailable")
    # ---- FIXED: arrays same length as `times`
    df = pd.DataFrame({
        "time_s": times,
        "avg_target": avg_target,
        "coverage_target": cov_target,
        "frac_overdose": frac_overdose,
        "total_mass_proxy": total_mass
    })
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries_fipy.csv"), index=False)

    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"[FiPy] Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"fipy_diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    return {"times": times, "df": df, "snapshots": snap_maps}

# %% [markdown]
# ## 6) Optional: tiny 3D diffusion (NumPy, constant D)

# %%
def simulate_diffusion_3d_numpy(cfg: DiffusionConfig):
    nx, ny, nz = cfg.nx//2, cfg.ny//2, cfg.nz
    Lx, Ly, Lz = cfg.Lx, cfg.Ly, cfg.Lz
    dx, dy, dz = Lx/(nx-1), Ly/(ny-1), Lz/(nz-1)
    C = np.zeros((nx, ny, nz), dtype=np.float64)
    D = cfg.D_grey
    dt = min(cfg.dt, 0.1 * min(dx,dy,dz)**2 / (6*D + 1e-30))
    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    x = np.linspace(0, Lx, nx); y = np.linspace(0, Ly, ny); z = np.linspace(0, Lz, nz)
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")
    cx, cy, cz = 0.25*Lx, 0.5*Ly, 0.5*Lz
    r = cfg.source_radius_frac * math.sqrt(Lx**2 + Ly**2 + Lz**2)
    src = (X-cx)**2 + (Y-cy)**2 + (Z-cz)**2 <= r**2
    if cfg.point_injection:
        C[src] += cfg.dose_amount

    def laplacian(C):
        Cp = np.pad(C, ((1,1),(1,1),(1,1)), mode="edge")
        d2x = (Cp[2:,1:-1,1:-1] - 2*Cp[1:-1,1:-1,1:-1] + Cp[:-2,1:-1,1:-1])/dx**2
        d2y = (Cp[1:-1,2:,1:-1] - 2*Cp[1:-1,1:-1,1:-1] + Cp[1:-1,:-2,1:-1])/dy**2
        d2z = (Cp[1:-1,1:-1,2:] - 2*Cp[1:-1,1:-1,1:-1] + Cp[1:-1,1:-1,:-2])/dz**2
        return d2x + d2y + d2z

    snapshots = {}
    snap_times = [0.0, cfg.t_final*0.5, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])

    for step, t in enumerate(times):
        if step in snap_indices:
            snapshots[float(f"{t:.3f}")] = C.copy()
        if step == nsteps:
            break
        L = laplacian(C)
        C_new = C + dt*(D*L - cfg.k_decay*C)
        if (not cfg.point_injection) and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            C_new[src] += cfg.infusion_rate * dt
        C = np.maximum(C_new, 0.0)

    for tkey, vol in snapshots.items():
        midz = vol[:,:,nz//2]
        plt.figure()
        plt.imshow(midz.T, origin="lower", extent=[0, Lx*1e3, 0, Ly*1e3], aspect="equal")
        plt.colorbar(label="C")
        plt.title(f"3D mid-slice at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"diffusion3d_slice_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    return {"times": times, "snapshots": snapshots}

# %% [markdown]
# ## 7) PK/PD ODEs + PD linkage

# %%
def pkpd_rhs(t, y, p: PKPDConfig, dose_fn: Callable[[float], float]):
    Cb, Cbr = y[0], y[1]
    input_rate = dose_fn(t)
    dCb = - (p.k_blood_to_brain + p.k_elim_blood + (p.k_blood_to_clear if p.three_compartment else 0.0)) * Cb \
          + p.k_brain_to_blood * Cbr \
          + input_rate / max(p.V_blood, 1e-12)
    dCbr = p.k_blood_to_brain * Cb - (p.k_brain_to_blood + p.k_deg_brain) * Cbr
    if p.three_compartment:
        Cl = y[2]
        dCl = p.k_blood_to_clear * Cb
        return [dCb, dCbr, dCl]
    else:
        return [dCb, dCbr]

def make_dose_fn(boluses: List[Tuple[float,float]], infusions: List[Tuple[float,float,float]]):
    boluses = boluses or []
    infusions = infusions or []
    def dose(t: float) -> float:
        rt = 0.0
        for tb, amt in boluses:
            if tb <= t < tb + 1.0:
                rt += amt
        for t_on, t_off, r in infusions:
            if t_on <= t <= t_off:
                rt += r
        return rt
    return dose

def simulate_pkpd(cfg: PKPDConfig):
    y0 = [0.0, 0.0] if not cfg.three_compartment else [0.0, 0.0, 0.0]
    t_eval = np.linspace(0, cfg.t_final, 1200)
    dose_fn = make_dose_fn(cfg.boluses, cfg.infusions)
    rhs = lambda t, y: pkpd_rhs(t, y, cfg, dose_fn)
    sol = solve_ivp(rhs, [0, cfg.t_final], y0, t_eval=t_eval, method="RK45", rtol=1e-6, atol=1e-9)

    if not sol.success:
        print("[WARN] PK/PD solver returned unsuccessful status.")

    Cb = sol.y[0]; Cbr = sol.y[1]
    Cl = sol.y[2] if cfg.three_compartment and sol.y.shape[0] > 2 else None

    E = cfg.Emax * (Cbr**cfg.hill_n) / (cfg.EC50**cfg.hill_n + Cbr**cfg.hill_n)

    dt = np.diff(sol.t, prepend=sol.t[0])
    time_above_ther = float(np.sum((Cbr >= cfg.therapeutic_thresh) * dt))
    time_over_toxic = float(np.sum((Cbr >= cfg.toxicity_thresh) * dt))
    auE = float(np.trapz(E, sol.t))
    total_dose = (sum([amt for _, amt in (cfg.boluses or [])]) +
                  sum([(t_off - t_on) * r for (t_on, t_off, r) in (cfg.infusions or [])]))

    if pd is None:
        raise RuntimeError("pandas unavailable")
    df = pd.DataFrame({"time_s": sol.t, "Cb": Cb, "Cbrain": Cbr, "Effect": E})
    df.to_csv(os.path.join(OUTDIR, "pkpd_timeseries.csv"), index=False)

    plt.figure()
    plt.plot(sol.t/3600.0, Cb, label="Blood")
    plt.plot(sol.t/3600.0, Cbr, label="Brain")
    plt.axhline(cfg.therapeutic_thresh, ls="--", lw=0.7, label="Therapeutic thresh")
    plt.axhline(cfg.toxicity_thresh, ls="--", lw=0.7, label="Toxicity thresh")
    plt.xlabel("Time (h)"); plt.ylabel("Concentration (arb.)")
    plt.title("PK across compartments"); plt.legend()
    plt.savefig(os.path.join(OUTDIR, "pkpd_compartments.png"), bbox_inches="tight"); plt.close()

    plt.figure()
    plt.plot(sol.t/3600.0, E)
    plt.xlabel("Time (h)"); plt.ylabel("Effect (PD, Hill)")
    plt.title("PD effect")
    plt.savefig(os.path.join(OUTDIR, "pkpd_effect.png"), bbox_inches="tight"); plt.close()

    return {
        "t": sol.t, "Cb": Cb, "Cbrain": Cbr, "Cl": Cl, "Effect": E,
        "time_above_ther": time_above_ther, "time_over_toxic": time_over_toxic,
        "auE": auE, "total_dose": total_dose
    }

# %% [markdown]
# ## 8) Optimization sketches (grid search + optional BO)

# %%
def score_objective(coverage_mean, overdose_mean, total_dose, w_cov=1.0, w_od=2.0, w_dose=0.1):
    return w_cov*coverage_mean - w_od*overdose_mean - w_dose*total_dose

def grid_search_pkpd(base_cfg: PKPDConfig, opt: OptimizeConfig):
    if pd is None:
        raise RuntimeError("pandas unavailable")
    records = []
    for amt in (opt.bolus_amounts or [0.5]):
        for interval_h in (opt.bolus_intervals_h or [6.0]):
            boluses = [(i*interval_h*3600.0, amt) for i in range(opt.n_bolus)]
            for inf_rate in (opt.infusion_rates or [0.0]):
                for inf_h in (opt.infusion_hours or [0.0]):
                    test = PKPDConfig(**asdict(base_cfg))
                    test.boluses = boluses
                    test.infusions = [(2*3600.0, (2+inf_h)*3600.0, inf_rate)] if (inf_h>0 and inf_rate>0) else []
                    res = simulate_pkpd(test)
                    cov = float(np.mean(res["Cbrain"] >= test.therapeutic_thresh))
                    od = float(np.mean(res["Cbrain"] >= test.toxicity_thresh))
                    score = score_objective(cov, od, res["total_dose"],
                                            opt.w_coverage, opt.w_overdose, opt.w_total_dose)
                    records.append({
                        "bolus_amt": amt, "bolus_interval_h": interval_h, "n_bolus": opt.n_bolus,
                        "inf_rate": inf_rate, "inf_hours": inf_h,
                        "coverage_mean": cov, "overdose_mean": od,
                        "total_dose": res["total_dose"], "score": score
                    })
    df = pd.DataFrame.from_records(records).sort_values("score", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "pkpd_grid_search_results.csv"), index=False)

    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (mean time frac ≥ therapeutic)")
    plt.ylabel("Overdose (mean time frac ≥ toxicity)")
    plt.title("PK/PD trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "pkpd_tradeoff_grid.png"), bbox_inches="tight"); plt.close()
    return df

def grid_search_diffusion(base_cfg: DiffusionConfig, opt: OptimizeConfig):
    if pd is None:
        raise RuntimeError("pandas unavailable")
    records = []
    for rate in (opt.dif_infusion_rates or [base_cfg.infusion_rate]):
        for dur in (opt.dif_infusion_durations or [base_cfg.infusion_t_off - base_cfg.infusion_t_on]):
            test = DiffusionConfig(**asdict(base_cfg))
            test.point_injection = False
            test.infusion_rate = rate
            test.infusion_t_on = 0.0
            test.infusion_t_off = dur
            out = simulate_diffusion_numpy(test)
            df_ts = out["df"]
            coverage_mean = float(df_ts["coverage_target"].mean())
            overdose_mean = float(df_ts["frac_overdose"].mean())
            total_dose = rate * dur * float(np.count_nonzero(out["source_mask"]))  # rough proxy
            score = score_objective(coverage_mean, overdose_mean, total_dose,
                                    opt.w_coverage, opt.w_overdose, opt.w_total_dose)
            records.append({
                "inf_rate": rate, "inf_dur_s": dur,
                "coverage_mean": coverage_mean, "overdose_mean": overdose_mean,
                "total_dose_proxy": total_dose, "score": score
            })
    df = pd.DataFrame.from_records(records).sort_values("score", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "diffusion_grid_search_results.csv"), index=False)

    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (spatial frac ≥ therapeutic)")
    plt.ylabel("Overdose (spatial frac ≥ toxicity)")
    plt.title("Diffusion trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "diffusion_tradeoff_grid.png"), bbox_inches="tight"); plt.close()
    return df

def bo_demo_pkpd(base_cfg: PKPDConfig, opt: OptimizeConfig, n_init=6, n_iter=12):
    if GaussianProcessRegressor is None:
        raise RuntimeError("scikit-learn not available; set try_bo=False or install sklearn.")
    space_bounds = np.array([[0.1, 1.5], [0.0, 2e-4]])  # bolus amt, inf_rate
    X, y = [], []
    kernel = C(1.0, (1e-2, 1e3)) * Matern(nu=1.5) + WhiteKernel(1e-5)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)

    def eval_point(amt, inf_rate):
        test = PKPDConfig(**asdict(base_cfg))
        test.boluses = [(0.0, amt), (6*3600.0, amt), (12*3600.0, amt)]
        test.infusions = [(2*3600.0, 6*3600.0, inf_rate)]
        res = simulate_pkpd(test)
        cov = float(np.mean(res["Cbrain"] >= test.therapeutic_thresh))
        od = float(np.mean(res["Cbrain"] >= test.toxicity_thresh))
        return score_objective(cov, od, res["total_dose"], opt.w_coverage, opt.w_overdose, opt.w_total_dose)

    for i in range(n_init):
        r = np.random.rand(2)
        x = space_bounds[:,0] + r*(space_bounds[:,1]-space_bounds[:,0])
        X.append(x); y.append(eval_point(x[0], x[1]))
    X, y = np.array(X), np.array(y)

    from scipy.stats import norm
    for it in range(n_iter):
        gp.fit(X, y)
        cand = space_bounds[:,0] + np.random.rand(200,2)*(space_bounds[:,1]-space_bounds[:,0])
        mu, sigma = gp.predict(cand, return_std=True)
        best = y.max()
        Z = (mu - best) / (sigma + 1e-9)
        ei = (mu - best) * norm.cdf(Z) + sigma * norm.pdf(Z)
        x_new = cand[np.argmax(ei)]
        y_new = eval_point(x_new[0], x_new[1])
        X = np.vstack([X, x_new]); y = np.append(y, y_new)

    df = pd.DataFrame({"bolus_amt": X[:,0], "inf_rate": X[:,1], "score": y}).sort_values("score", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "pkpd_bo_results.csv"), index=False)
    return df

# %% [markdown]
# ## 9) Run: Diffusion, PK/PD, Optimization

# %%
print("Running diffusion (NumPy)...")
dif_out = simulate_diffusion_numpy(diff_cfg)

if diff_cfg.use_fipy and fp is not None:
    print("Running diffusion (FiPy)...")
    _ = simulate_diffusion_fipy(diff_cfg)

if diff_cfg.do_3d:
    print("Running tiny 3D diffusion...")
    _ = simulate_diffusion_3d_numpy(diff_cfg)

print("Running PK/PD...")
pk_out = simulate_pkpd(pkpd_cfg)

print("Running grid searches...")
pk_grid = grid_search_pkpd(pkpd_cfg, opt_cfg) if opt_cfg.enable_pkpd_grid else None
dif_grid = grid_search_diffusion(diff_cfg, opt_cfg) if opt_cfg.enable_diffusion_grid else None

if opt_cfg.try_bo:
    print("Running simple BO demo for PK/PD...")
    _ = bo_demo_pkpd(pkpd_cfg, opt_cfg)

# %% [markdown]
# ## 10) Notebook footer: outputs list & related work

# %%
print("Saved files in:", OUTDIR)
print(sorted(os.listdir(OUTDIR)))

print("\nRelated work:")
print(" - FiPy: A Finite Volume PDE Solver in Python. Docs: fipy.readthedocs.io")
print(" - eBrain / virtual-brain-style in-silico platforms for neuro drug delivery & transport simulations.")

Configs loaded.
Running diffusion (NumPy)...
Running PK/PD...
Running grid searches...
Saved files in: /kaggle/working
['.virtual_documents', 'diffusion_grid_search_results.csv', 'diffusion_heatmap_t0p0sppng.png', 'diffusion_heatmap_t1800p0sppng.png', 'diffusion_heatmap_t2700p0sppng.png', 'diffusion_heatmap_t3600p0sppng.png', 'diffusion_heatmap_t900p0sppng.png', 'diffusion_metrics_timeseries.png', 'diffusion_timeseries.csv', 'diffusion_tradeoff_grid.png', 'pkpd_compartments.png', 'pkpd_effect.png', 'pkpd_grid_search_results.csv', 'pkpd_timeseries.csv', 'pkpd_tradeoff_grid.png']

Related work:
 - FiPy: A Finite Volume PDE Solver in Python. Docs: fipy.readthedocs.io
 - eBrain / virtual-brain-style in-silico platforms for neuro drug delivery & transport simulations.


In [1]:
# %% [markdown]
# # In-Silico Brain Drug Delivery (Diffusion + PK/PD) — Updated Run
# CPU-only; let me know if/when we move to CuPy/PyTorch for GPU.

# %%
import os, math, json, time
from dataclasses import dataclass, asdict
from typing import Tuple, Optional, Dict, List, Callable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

try:
    import nibabel as nib
except Exception:
    nib = None
try:
    import fipy as fp
except Exception:
    fp = None

np.random.seed(42)
OUTDIR = "/kaggle/working"
os.makedirs(OUTDIR, exist_ok=True)
plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.dpi"] = 140

# ===============================
# 1) CONFIG
# ===============================
@dataclass
class DiffusionConfig:
    # Numerics / domain
    use_fipy: bool = False
    do_3d: bool = False
    nx: int = 128
    ny: int = 128
    nz: int = 32
    Lx: float = 0.032
    Ly: float = 0.032
    Lz: float = 0.016
    dt: float = 0.25
    t_final: float = 3600.0
    bctype: str = "neumann"  # "neumann" or "dirichlet"

    # Tissue parameters
    D_white: float = 5e-10
    D_grey: float = 1.0e-9
    k_decay: float = 1.0e-4

    # Geometry (now configurable)
    src_x_frac: float = 0.25
    src_y_frac: float = 0.50
    tgt_x_frac: float = 0.35   # moved closer by default so coverage can be observed in 1h
    tgt_y_frac: float = 0.50
    source_radius_frac: float = 0.02
    target_radius_frac: float = 0.10

    # Dosing (PDE)
    point_injection: bool = False         # use continuous infusion by default
    dose_amount: float = 1.0              # if point injection
    infusion_rate: float = 3e-3           # stronger but still demo-level
    infusion_t_on: float = 0.0
    infusion_t_off: float = 3600.0

    # Thresholds (for coverage/overdose in PDE)
    therapeutic_thresh: float = 0.01
    toxicity_thresh: float = 0.5

    # Optional NIfTI tissue mask
    use_nifti_mask: bool = False
    nifti_path: Optional[str] = None

@dataclass
class PKPDConfig:
    three_compartment: bool = True
    # Rates (1/s)
    k_blood_to_brain: float = 8e-4
    k_brain_to_blood: float = 5e-4
    k_elim_blood: float = 3e-4
    k_deg_brain: float = 2e-4
    k_blood_to_clear: float = 0.0
    # Volumes (arb. but consistent)
    V_blood: float = 5.0
    V_brain: float = 1.2
    V_clear: float = 2.0
    # Dosing (ODE)
    boluses: List[Tuple[float, float]] = None
    infusions: List[Tuple[float, float, float]] = None
    t_final: float = 24*3600.0
    # PD
    EC50: float = 0.2
    Emax: float = 1.0
    hill_n: float = 2.0
    therapeutic_thresh: float = 0.05   # lowered a bit for exploratory signal
    toxicity_thresh: float = 1.0

@dataclass
class OptimizeConfig:
    enable_pkpd_grid: bool = True
    enable_diffusion_grid: bool = True
    # PK/PD grid ranges
    bolus_amounts: List[float] = None
    bolus_intervals_h: List[float] = None
    n_bolus: int = 3
    infusion_rates: List[float] = None
    infusion_hours: List[float] = None
    # Diffusion grid ranges (continuous infusion)
    dif_infusion_rates: List[float] = None
    dif_infusion_durations: List[float] = None
    # Objective weights
    w_coverage: float = 1.0
    w_overdose: float = 2.0
    w_total_dose: float = 0.1
    try_bo: bool = False  # keep off unless sklearn is installed

def default_configs():
    diff = DiffusionConfig()
    pkpd = PKPDConfig(
        three_compartment=True,
        boluses=[(0.0, 0.6), (6*3600.0, 0.6)],
        infusions=[(2*3600.0, 8*3600.0, 2e-4)]
    )
    opt = OptimizeConfig(
        bolus_amounts=[0.4, 0.6, 0.8, 1.0],
        bolus_intervals_h=[4.0, 6.0],
        n_bolus=3,
        infusion_rates=[0.0, 1e-4, 2e-4, 5e-4],
        infusion_hours=[0.0, 4.0, 8.0],
        dif_infusion_rates=[1e-3, 2e-3, 3e-3],
        dif_infusion_durations=[1800.0, 3600.0, 7200.0],
    )
    return diff, pkpd, opt

diff_cfg, pkpd_cfg, opt_cfg = default_configs()
print("Configs loaded (updated defaults).")

# ===============================
# 2) GEOMETRY / MASKS
# ===============================
def make_2d_masks(nx, ny, Lx, Ly, source_radius_frac, target_radius_frac,
                  src_x_frac, src_y_frac, tgt_x_frac, tgt_y_frac):
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)
    X, Y = np.meshgrid(x, y, indexing="ij")

    # Central oval ≈ grey matter
    cx, cy = Lx/2, Ly/2
    a, b = 0.35*Lx, 0.35*Ly
    gm_mask = ((X - cx)**2 / a**2 + (Y - cy)**2 / b**2) <= 1.0
    wm_mask = ~gm_mask

    # Source disk
    sx, sy = src_x_frac*Lx, src_y_frac*Ly
    diag = math.hypot(Lx, Ly)
    r_src = source_radius_frac * diag
    source_mask = (X - sx)**2 + (Y - sy)**2 <= r_src**2

    # Target disk
    tx, ty = tgt_x_frac*Lx, tgt_y_frac*Ly
    r_tgt = target_radius_frac * diag
    target_mask = (X - tx)**2 + (Y - ty)**2 <= r_tgt**2

    return gm_mask, wm_mask, source_mask, target_mask, (X, Y)

def build_D_map(gm_mask, wm_mask, D_grey, D_white):
    return np.where(gm_mask, D_grey, D_white).astype(np.float64)

# ===============================
# 3) DIFFUSION SOLVER (NumPy)
# ===============================
def divergence_of_D_gradC(C, D, dx, dy, bctype="neumann"):
    C = C.astype(np.float64)
    D = D.astype(np.float64)

    if bctype.lower() == "neumann":
        C_pad = np.pad(C, ((1,1),(1,1)), mode="edge")
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")
    else:  # Dirichlet zero
        C_pad = np.pad(C, ((1,1),(1,1)), mode="constant", constant_values=0.0)
        D_pad = np.pad(D, ((1,1),(1,1)), mode="edge")

    # face-averaged D
    Dxp = 0.5*(D_pad[2:,1:-1] + D_pad[1:-1,1:-1])
    Dxm = 0.5*(D_pad[1:-1,1:-1] + D_pad[0:-2,1:-1])
    Dyp = 0.5*(D_pad[1:-1,2:] + D_pad[1:-1,1:-1])
    Dym = 0.5*(D_pad[1:-1,1:-1] + D_pad[1:-1,0:-2])

    dCdx_p = (C_pad[2:,1:-1] - C_pad[1:-1,1:-1])
    dCdx_m = (C_pad[1:-1,1:-1] - C_pad[0:-2,1:-1])
    dCdy_p = (C_pad[1:-1,2:] - C_pad[1:-1,1:-1])
    dCdy_m = (C_pad[1:-1,1:-1] - C_pad[1:-1,0:-2])

    Fx_p = Dxp * dCdx_p
    Fx_m = Dxm * dCdx_m
    Fy_p = Dyp * dCdy_p
    Fy_m = Dym * dCdy_m

    # Note: since Fx_* omit /dx, divide by dx**2 here (and similarly for y)
    return (Fx_p - Fx_m)/(dx**2) + (Fy_p - Fy_m)/(dy**2)

def simulate_diffusion_numpy(cfg: DiffusionConfig):
    nx, ny = cfg.nx, cfg.ny
    dx, dy = cfg.Lx/(nx-1), cfg.Ly/(ny-1)

    gm_mask, wm_mask, source_mask, target_mask, (X, Y) = make_2d_masks(
        nx, ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac,
        cfg.src_x_frac, cfg.src_y_frac, cfg.tgt_x_frac, cfg.tgt_y_frac
    )

    if cfg.use_nifti_mask and cfg.nifti_path and nib is not None:
        try:
            img = nib.load(cfg.nifti_path)
            data = img.get_fdata()
            sl = data.shape[2]//2
            vol = data[:,:,sl]
            xi = (np.linspace(0, vol.shape[0]-1, nx)).astype(int)
            yi = (np.linspace(0, vol.shape[1]-1, ny)).astype(int)
            vol_res = vol[np.ix_(xi, yi)]
            gm_mask = (vol_res > 0.5)
            wm_mask = ~gm_mask
        except Exception as e:
            print("NIfTI load/resample failed, using synthetic mask.", e)

    D = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white)

    dt_stable = 0.2 * min(dx, dy)**2 / (4.0 * max(cfg.D_grey, cfg.D_white) + 1e-30)
    dt = min(cfg.dt, dt_stable)
    if cfg.dt > dt_stable:
        print(f"[WARN] dt={cfg.dt:.3e} > stability≈{dt_stable:.3e}. Using dt={dt:.3e}")

    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    C = np.zeros((nx, ny), dtype=np.float64)
    if cfg.point_injection and np.count_nonzero(source_mask) > 0:
        C[source_mask] += cfg.dose_amount

    infusion_active = (not cfg.point_injection)

    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []
    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        if step in snap_indices:
            snap_maps[float(f"{t:.3f}")] = C.copy()

        tgt_vals = C[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(C >= cfg.toxicity_thresh)))
        total_mass.append(float(C.mean()))

        if step == nsteps:
            break

        S = 0.0
        if infusion_active and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            S = cfg.infusion_rate

        div = divergence_of_D_gradC(C, D, dx, dy, cfg.bctype)
        C_new = C + dt*(div - cfg.k_decay*C)
        if S != 0.0:
            C_new[source_mask] += S * dt
        C = np.maximum(C_new, 0.0)

    df = pd.DataFrame({
        "time_s": times,
        "avg_target": avg_target,
        "coverage_target": cov_target,
        "frac_overdose": frac_overdose,
        "total_mass_proxy": total_mass
    })
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries.csv"), index=False)

    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    plt.figure()
    plt.plot(df["time_s"]/60.0, df["avg_target"], label="Avg target conc")
    plt.plot(df["time_s"]/60.0, df["coverage_target"], label="Coverage (target ≥ thresh)")
    plt.plot(df["time_s"]/60.0, df["frac_overdose"], label="Overdose fraction")
    plt.xlabel("Time (min)"); plt.legend(); plt.title("Diffusion metrics over time")
    plt.savefig(os.path.join(OUTDIR, "diffusion_metrics_timeseries.png"), bbox_inches="tight"); plt.close()

    return {"times": times, "df": df, "snapshots": snap_maps,
            "target_mask": target_mask, "source_mask": source_mask}

# (Optional) FiPy variant (unchanged numerics, used only if cfg.use_fipy=True and FiPy available)
def simulate_diffusion_fipy(cfg: DiffusionConfig):
    if fp is None:
        raise RuntimeError("FiPy not available.")
    mesh = fp.Grid2D(nx=cfg.nx, ny=cfg.ny, dx=cfg.Lx/(cfg.nx-1), dy=cfg.Ly/(cfg.ny-1))
    Cvar = fp.CellVariable(mesh=mesh, name="C", value=0.0)

    gm_mask, wm_mask, source_mask, target_mask, _ = make_2d_masks(
        cfg.nx, cfg.ny, cfg.Lx, cfg.Ly, cfg.source_radius_frac, cfg.target_radius_frac,
        cfg.src_x_frac, cfg.src_y_frac, cfg.tgt_x_frac, cfg.tgt_y_frac
    )
    D_map = build_D_map(gm_mask, wm_mask, cfg.D_grey, cfg.D_white).ravel(order='F')
    D = fp.CellVariable(mesh=mesh, value=D_map)

    if cfg.point_injection:
        cur = Cvar.value.copy()
        cur[source_mask.ravel(order='F')] += cfg.dose_amount
        Cvar.setValue(cur)

    eq = fp.TransientTerm() == fp.DiffusionTerm(coeff=D) - cfg.k_decay * Cvar
    dt = cfg.dt
    nsteps = int(np.ceil(cfg.t_final / dt))
    times = np.linspace(0, cfg.t_final, nsteps+1)

    avg_target, cov_target, frac_overdose, total_mass = [], [], [], []
    snap_times = [0.0, cfg.t_final*0.25, cfg.t_final*0.5, cfg.t_final*0.75, cfg.t_final]
    snap_indices = set([int(round(t / dt)) for t in snap_times])
    snap_maps = {}

    for step, t in enumerate(times):
        if step in snap_indices:
            Cmap = Cvar.value.reshape(cfg.nx, cfg.ny, order='F').copy()
            snap_maps[float(f"{t:.3f}")] = Cmap

        arr = Cvar.value.reshape(cfg.nx, cfg.ny, order='F')
        tgt_vals = arr[target_mask]
        avg_target.append(float(np.mean(tgt_vals)))
        cov_target.append(float(np.mean(tgt_vals >= cfg.therapeutic_thresh)))
        frac_overdose.append(float(np.mean(arr >= cfg.toxicity_thresh)))
        total_mass.append(float(arr.mean()))

        if step == nsteps:
            break

        if (not cfg.point_injection) and (cfg.infusion_t_on <= t <= cfg.infusion_t_off):
            cur = Cvar.value.copy()
            cur[source_mask.ravel(order='F')] += cfg.infusion_rate * dt
            Cvar.setValue(cur)

        eq.solve(var=Cvar, dt=dt)

    df = pd.DataFrame({
        "time_s": times,
        "avg_target": avg_target,
        "coverage_target": cov_target,
        "frac_overdose": frac_overdose,
        "total_mass_proxy": total_mass
    })
    df.to_csv(os.path.join(OUTDIR, "diffusion_timeseries_fipy.csv"), index=False)

    for tkey, Cmap in snap_maps.items():
        plt.figure()
        plt.imshow(Cmap.T, origin="lower", extent=[0, cfg.Lx*1e3, 0, cfg.Ly*1e3], aspect="equal")
        plt.colorbar(label="Concentration (arb.)")
        plt.contour(target_mask.T, levels=[0.5], colors='white', linewidths=0.7)
        plt.title(f"[FiPy] Diffusion heatmap at t={float(tkey)/60:.1f} min")
        plt.xlabel("x (mm)"); plt.ylabel("y (mm)")
        fn = f"fipy_diffusion_heatmap_t{float(tkey):.1f}s.png".replace(".", "p")
        plt.savefig(os.path.join(OUTDIR, fn), bbox_inches="tight"); plt.close()

    return {"times": times, "df": df, "snapshots": snap_maps}

# ===============================
# 4) PK/PD + PD
# ===============================
def pkpd_rhs(t, y, p: PKPDConfig, dose_fn: Callable[[float], float]):
    Cb, Cbr = y[0], y[1]
    input_rate = dose_fn(t)
    dCb = - (p.k_blood_to_brain + p.k_elim_blood + (p.k_blood_to_clear if p.three_compartment else 0.0)) * Cb \
          + p.k_brain_to_blood * Cbr \
          + input_rate / max(p.V_blood, 1e-12)
    dCbr = p.k_blood_to_brain * Cb - (p.k_brain_to_blood + p.k_deg_brain) * Cbr
    if p.three_compartment:
        Cl = y[2]
        dCl = p.k_blood_to_clear * Cb
        return [dCb, dCbr, dCl]
    else:
        return [dCb, dCbr]

def make_dose_fn(boluses: List[Tuple[float,float]], infusions: List[Tuple[float,float,float]]):
    boluses = boluses or []
    infusions = infusions or []
    def dose(t: float) -> float:
        rt = 0.0
        for tb, amt in boluses:
            if tb <= t < tb + 1.0:   # 1-second square for bolus
                rt += amt
        for t_on, t_off, r in infusions:
            if t_on <= t <= t_off:
                rt += r
        return rt
    return dose

def simulate_pkpd(cfg: PKPDConfig):
    y0 = [0.0, 0.0] if not cfg.three_compartment else [0.0, 0.0, 0.0]
    t_eval = np.linspace(0, cfg.t_final, 1200)
    dose_fn = make_dose_fn(cfg.boluses, cfg.infusions)
    rhs = lambda t, y: pkpd_rhs(t, y, cfg, dose_fn)
    sol = solve_ivp(rhs, [0, cfg.t_final], y0, t_eval=t_eval, method="RK45", rtol=1e-6, atol=1e-9)

    Cb = sol.y[0]; Cbr = sol.y[1]
    Cl = sol.y[2] if cfg.three_compartment and sol.y.shape[0] > 2 else None

    # PD (Hill)
    E = cfg.Emax * (Cbr**cfg.hill_n) / (cfg.EC50**cfg.hill_n + Cbr**cfg.hill_n)

    # KPIs
    dt = np.diff(sol.t, prepend=sol.t[0])
    time_above_ther = float(np.sum((Cbr >= cfg.therapeutic_thresh) * dt))
    time_over_toxic = float(np.sum((Cbr >= cfg.toxicity_thresh) * dt))
    auE = float(np.trapz(E, sol.t))
    total_dose = (sum([amt for _, amt in (cfg.boluses or [])]) +
                  sum([(t_off - t_on) * r for (t_on, t_off, r) in (cfg.infusions or [])]))

    df = pd.DataFrame({"time_s": sol.t, "Cb": Cb, "Cbrain": Cbr, "Effect": E})
    df.to_csv(os.path.join(OUTDIR, "pkpd_timeseries.csv"), index=False)

    plt.figure()
    plt.plot(sol.t/3600.0, Cb, label="Blood")
    plt.plot(sol.t/3600.0, Cbr, label="Brain")
    plt.axhline(cfg.therapeutic_thresh, ls="--", lw=0.7, label="Therapeutic thresh")
    plt.axhline(cfg.toxicity_thresh, ls="--", lw=0.7, label="Toxicity thresh")
    plt.xlabel("Time (h)"); plt.ylabel("Concentration (arb.)")
    plt.title("PK across compartments"); plt.legend()
    plt.savefig(os.path.join(OUTDIR, "pkpd_compartments.png"), bbox_inches="tight"); plt.close()

    plt.figure()
    plt.plot(sol.t/3600.0, E)
    plt.xlabel("Time (h)"); plt.ylabel("Effect (PD, Hill)")
    plt.title("PD effect")
    plt.savefig(os.path.join(OUTDIR, "pkpd_effect.png"), bbox_inches="tight"); plt.close()

    return {
        "t": sol.t, "Cb": Cb, "Cbrain": Cbr, "Cl": Cl, "Effect": E,
        "time_above_ther": time_above_ther, "time_over_toxic": time_over_toxic,
        "auE": auE, "total_dose": total_dose
    }

# ===============================
# 5) OPTIMIZATION SKETCHES
# ===============================
def score_objective(coverage_mean, overdose_mean, total_dose, w_cov=1.0, w_od=2.0, w_dose=0.1):
    return w_cov*coverage_mean - w_od*overdose_mean - w_dose*total_dose

def grid_search_pkpd(base_cfg: PKPDConfig, opt: OptimizeConfig):
    records = []
    for amt in (opt.bolus_amounts or [0.6]):
        for interval_h in (opt.bolus_intervals_h or [6.0]):
            boluses = [(i*interval_h*3600.0, amt) for i in range(opt.n_bolus)]
            for inf_rate in (opt.infusion_rates or [0.0]):
                for inf_h in (opt.infusion_hours or [0.0]):
                    test = PKPDConfig(**asdict(base_cfg))
                    test.boluses = boluses
                    test.infusions = [(2*3600.0, (2+inf_h)*3600.0, inf_rate)] if (inf_h>0 and inf_rate>0) else []
                    res = simulate_pkpd(test)
                    cov = float(np.mean(res["Cbrain"] >= test.therapeutic_thresh))
                    od = float(np.mean(res["Cbrain"] >= test.toxicity_thresh))
                    score = score_objective(cov, od, res["total_dose"],
                                            opt.w_coverage, opt.w_overdose, opt.w_total_dose)
                    records.append({
                        "bolus_amt": amt, "bolus_interval_h": interval_h, "n_bolus": opt.n_bolus,
                        "inf_rate": inf_rate, "inf_hours": inf_h,
                        "coverage_mean": cov, "overdose_mean": od,
                        "total_dose": res["total_dose"], "score": score
                    })
    df = pd.DataFrame.from_records(records).sort_values("score", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "pkpd_grid_search_results.csv"), index=False)

    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (mean time frac ≥ therapeutic)")
    plt.ylabel("Overdose (mean time frac ≥ toxicity)")
    plt.title("PK/PD trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "pkpd_tradeoff_grid.png"), bbox_inches="tight"); plt.close()
    return df

def grid_search_diffusion(base_cfg: DiffusionConfig, opt: OptimizeConfig):
    records = []
    for rate in (opt.dif_infusion_rates or [base_cfg.infusion_rate]):
        for dur in (opt.dif_infusion_durations or [base_cfg.infusion_t_off - base_cfg.infusion_t_on]):
            test = DiffusionConfig(**asdict(base_cfg))
            test.point_injection = False
            test.infusion_rate = rate
            test.infusion_t_on = 0.0
            test.infusion_t_off = dur
            out = simulate_diffusion_numpy(test)
            df_ts = out["df"]
            coverage_mean = float(df_ts["coverage_target"].mean())
            overdose_mean = float(df_ts["frac_overdose"].mean())
            total_dose = rate * dur * float(np.count_nonzero(out["source_mask"]))  # rough proxy
            score = score_objective(coverage_mean, overdose_mean, total_dose,
                                    opt.w_coverage, opt.w_overdose, opt.w_total_dose)
            records.append({
                "inf_rate": rate, "inf_dur_s": dur,
                "coverage_mean": coverage_mean, "overdose_mean": overdose_mean,
                "total_dose_proxy": total_dose, "score": score
            })
    df = pd.DataFrame.from_records(records).sort_values("score", ascending=False)
    df.to_csv(os.path.join(OUTDIR, "diffusion_grid_search_results.csv"), index=False)

    plt.figure()
    plt.scatter(df["coverage_mean"], df["overdose_mean"], s=30)
    plt.xlabel("Coverage (spatial frac ≥ therapeutic)")
    plt.ylabel("Overdose (spatial frac ≥ toxicity)")
    plt.title("Diffusion trade-off grid")
    plt.savefig(os.path.join(OUTDIR, "diffusion_tradeoff_grid.png"), bbox_inches="tight"); plt.close()
    return df

# ===============================
# 6) RUN
# ===============================
print("Running diffusion (NumPy)…")
dif_out = simulate_diffusion_numpy(diff_cfg)

if diff_cfg.use_fipy and fp is not None:
    print("Running diffusion (FiPy)…")
    _ = simulate_diffusion_fipy(diff_cfg)

if diff_cfg.do_3d:
    print("Tiny 3D run is disabled by default to keep CPU fast.")

print("Running PK/PD…")
pk_out = simulate_pkpd(pkpd_cfg)

print("Running grid searches…")
pk_grid = grid_search_pkpd(pkpd_cfg, opt_cfg) if opt_cfg.enable_pkpd_grid else None
dif_grid = grid_search_diffusion(diff_cfg, opt_cfg) if opt_cfg.enable_diffusion_grid else None

print("Saved files in:", OUTDIR)
print(sorted(os.listdir(OUTDIR)))

print("\nRelated work:")
print(" - FiPy: A Finite Volume PDE Solver in Python. Docs: fipy.readthedocs.io")
print(" - eBrain / virtual-brain-style in-silico platforms for neuro drug delivery & transport simulations.")

Configs loaded (updated defaults).
Running diffusion (NumPy)…
Running PK/PD…
Running grid searches…
Saved files in: /kaggle/working
['.virtual_documents', 'diffusion_grid_search_results.csv', 'diffusion_heatmap_t0p0sppng.png', 'diffusion_heatmap_t1800p0sppng.png', 'diffusion_heatmap_t2700p0sppng.png', 'diffusion_heatmap_t3600p0sppng.png', 'diffusion_heatmap_t900p0sppng.png', 'diffusion_metrics_timeseries.png', 'diffusion_timeseries.csv', 'diffusion_tradeoff_grid.png', 'pkpd_compartments.png', 'pkpd_effect.png', 'pkpd_grid_search_results.csv', 'pkpd_timeseries.csv', 'pkpd_tradeoff_grid.png']

Related work:
 - FiPy: A Finite Volume PDE Solver in Python. Docs: fipy.readthedocs.io
 - eBrain / virtual-brain-style in-silico platforms for neuro drug delivery & transport simulations.
